# Analyse imagePoints

Our aim is to calibrate the cameras against the laser positions (3D) and projections of lasers into camrea images (as 2D image points where lines converge).

During this stage we want to:

1. Deselect any beamcaptures that are wrongly detected
2. Deselect any lasercaptures where the convergence point does not follow the circle in Figure 2 (NB : these should be re-selected after completing the camera calibration stage)
3. Evaluate residuals using Figure 3

In [69]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

In [70]:
with open('../../Application/bin/data/Patch.json') as file:
	patch = json.load(file)
	

In [71]:
def find_node_by_name(node_name):
	for node_id in patch['Nodes']:
		if patch['Nodes'][node_id]['Name'] == node_name:
			return patch['Nodes'][node_id]
	raise(Exception("Node not found"))

In [72]:
calibrate_node = find_node_by_name('Calibrate')

Here you might need to manually select which camera index you're looking at

In [73]:
first_camera_index = 2
first_camera = calibrate_node['Content']['cameraCaptures']['captures'][first_camera_index]

In [74]:
#gather laser image points function
def gather_laser_image_points(camera_capture):
	laser_image_points = []
	laser_names = []
	for laser_capture in first_camera['laserCaptures']['captures']:
		if laser_capture['Selected']:
			laser_image_points.append(laser_capture['imagePointInCamera'])
			laser_names.append(laser_capture['serialNumber'])
	return laser_image_points, laser_names

In [75]:
laser_image_points, laser_names = gather_laser_image_points(first_camera)
data = np.array(laser_image_points)

# Figure 1

In [76]:
fig = px.scatter(x=data[:,0], y=data[:,1],hover_name=laser_names)
fig.show()

If you have trouble calibrating cameras, then try the following:

* Use intrinsic guess and set an initial postiion 
* Fix the lasers and use Bundle adjust lasers instead (needs a rough initial camera position) - note that you'll need more than one camera selected at a time since bundle adjustment rejects single camera scenarios



In [77]:
def get_laser_capture(camera_capture, serialNumber):
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if int(laser_capture['serialNumber']) == serialNumber:
			return laser_capture

get_laser_capture(first_camera, 22).keys()

dict_keys(['Color', 'Selected', 'beamCaptures', 'date', 'directory', 'imagePointInCamera', 'linesWithCommonPointSolveResult', 'parameters', 'serialNumber', 'time', 'timestamp'])

In [78]:
# gather data
data_points = []
for camera_capture in calibrate_node['Content']['cameraCaptures']['captures']:
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if laser_capture['Selected']:
			data_point = {}
			data_point['Camera'] = "{0} {1}".format(camera_capture['date'], camera_capture['time'])
			data_point['x'] = laser_capture['imagePointInCamera'][0]
			data_point['y'] = laser_capture['imagePointInCamera'][1]
			data_point['Laser'] = "Laser {0}".format(laser_capture['serialNumber'])
			
			if 'residual' in data_point:
				data_point['residual'] = laser_capture['serialNumber']
			else:
				data_point['residual'] = 0
				
			#count active beam captures
			active_beam_capture_count = 0
			for beam_capture in laser_capture['beamCaptures']['captures']:
				if beam_capture['Selected']:
					active_beam_capture_count = active_beam_capture_count + 1
			data_point['Active beam captures'] = active_beam_capture_count
			
			data_points.append(data_point)



# Figure 2

Disable LaserCaptures until we have good looking ellipses below:

Then we keep solving the intiial camera calibration, progressively removiving points with largest values until we get a good camera solve (e.g. values > 20k can be confusing for residuals)

In [79]:
fig = px.scatter(data_points
	, x='x'
	, y='y'
	, hover_name='Laser'
	, hover_data=['Active beam captures']
	, color='Camera')
fig.update_yaxes(autorange="reversed")
fig.show()

In [80]:
for data_point in data_points:
	if data_point['x'] == 0:
		print(data_point)

{'Camera': 'Thu 2024.11.21 03:25', 'x': 0.0, 'y': 0.0, 'Laser': 'Laser 19', 'residual': 0, 'Active beam captures': 1}


In [81]:
calibrate_node['Content']['cameraCaptures']['captures'][0]['date']

'Wed 2024.11.20'

# Figure 3

Two figures below show where most of the errors are.

Generally we use the `Prune/Beam captures by residual` function on the Calibrate node (e.g. with a value of 1000px).

Note that all camera captures are shown, even if they are deselected.

check
https://paper.dropbox.com/doc/KC68KC72-Calibration-log-v2--Bh4BhpiBl8CL5XP2Buzoq82GAg-zJiuoF5tDxpK6u18Sw23M#:uid=745312935561881238403951&h2=Athens-night-calibration-log


In [82]:
# get residuals on beamCaptures

data_points = []
for camera_capture in calibrate_node['Content']['cameraCaptures']['captures']:
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if laser_capture['Selected']:
			beam_capture_index = 0
			for beam_capture in laser_capture['beamCaptures']['captures']:
				if beam_capture['Selected']:
					data_point = {}
					data_point['Camera'] = "{0} {1}".format(camera_capture['date'], camera_capture['time'])
					data_point['Laser'] = laser_capture['serialNumber']
					data_point['projectionPoint_x'] = beam_capture['projectionPoint'][0]
					data_point['projectionPoint_y'] = beam_capture['projectionPoint'][1]
					data_point['residual'] = beam_capture['residual']
					data_point['beam_capture_index'] = beam_capture_index
					data_points.append(data_point)
					beam_capture_index += 1


fig = px.bar(data_points
	, x='Camera'
	, color='Laser'
	, y='residual'
	, hover_name='Laser'
	, hover_data=['beam_capture_index', 'projectionPoint_x', 'projectionPoint_y']
	, barmode="group")
fig.update_yaxes(autorange="reversed")
fig.show()

In [83]:
residuals = []
for data_point in data_points:
    residuals.append(data_point['residual'])
mean_residual = np.mean(residuals)

fig = px.scatter(data_points
    , x='residual'
    , y='Laser'
    , color='Camera'
    , hover_name='Laser'
	, hover_data=['beam_capture_index', 'projectionPoint_x', 'projectionPoint_y']
    , title="Mean residual {0} px".format(mean_residual))
fig.show()

In [84]:
select_laser = 26
data_points_selected = list(filter(lambda data_point: data_point['Laser'] == select_laser, data_points))


fig = px.scatter_3d(data_points_selected
    , x='projectionPoint_x'
    , y='projectionPoint_y'
    , z='residual'
    , color='Camera'
    , hover_name='Laser'
	, hover_data=['beam_capture_index', 'projectionPoint_x', 'projectionPoint_y']
    , title="Residuals for selected laser #{0}".format(select_laser))
fig.show()

In [85]:
data_points_selected

[{'Camera': 'Wed 2024.11.20 23:51',
  'Laser': 26,
  'projectionPoint_x': 0.5,
  'projectionPoint_y': -0.5,
  'residual': 0.8677765727043152,
  'beam_capture_index': 0},
 {'Camera': 'Wed 2024.11.20 23:51',
  'Laser': 26,
  'projectionPoint_x': -0.5,
  'projectionPoint_y': 0.5,
  'residual': 0.6886775493621826,
  'beam_capture_index': 1},
 {'Camera': 'Wed 2024.11.20 23:51',
  'Laser': 26,
  'projectionPoint_x': 0.5,
  'projectionPoint_y': 0.5,
  'residual': 0.15760403871536255,
  'beam_capture_index': 2},
 {'Camera': 'Thu 2024.11.21 00:00',
  'Laser': 26,
  'projectionPoint_x': -0.5,
  'projectionPoint_y': -0.5,
  'residual': 0.4450315535068512,
  'beam_capture_index': 0},
 {'Camera': 'Thu 2024.11.21 00:00',
  'Laser': 26,
  'projectionPoint_x': 0.5,
  'projectionPoint_y': -0.5,
  'residual': 0.8847813010215759,
  'beam_capture_index': 1},
 {'Camera': 'Thu 2024.11.21 00:00',
  'Laser': 26,
  'projectionPoint_x': -0.5,
  'projectionPoint_y': 0.5,
  'residual': 0.5899984240531921,
  'beam